# Objetivos do trabalho

* Qual time tem os melhores jogadores de defesa, meio-campo e ataque?
* Existe realmente uma vantagem do time da casa?
* As equipes com jogadores mais ofensivos marcam mais gols? E times com goleiros mais fortes recebem menos gols?
* Qual time tem a maior sequência de vitórias?
* O melhor time sempre vence? Você pode explicar por que um time mais fraco às vezes ganha?

In [1]:
import re

import numpy as np
import pandas as pd

# EDA

In [2]:
data = pd.read_csv("international_matches.csv", parse_dates=["date"])

In [3]:
data.head()

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
0,1993-08-08,Bolivia,Uruguay,South America,South America,59,22,0,0,3,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993-08-08,Brazil,Mexico,South America,North America,8,14,0,0,1,...,No,Draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-08-08,Ecuador,Venezuela,South America,South America,35,94,0,0,5,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993-08-08,Guinea,Sierra Leone,Africa,Africa,65,86,0,0,1,...,No,Win,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993-08-08,Paraguay,Argentina,South America,South America,67,5,0,0,1,...,No,Lose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.isna().sum()

date                                 0
home_team                            0
away_team                            0
home_team_continent                  0
away_team_continent                  0
home_team_fifa_rank                  0
away_team_fifa_rank                  0
home_team_total_fifa_points          0
away_team_total_fifa_points          0
home_team_score                      0
away_team_score                      0
tournament                           0
city                                 0
country                              0
neutral_location                     0
shoot_out                            0
home_team_result                     0
home_team_goalkeeper_score       15542
away_team_goalkeeper_score       15826
home_team_mean_defense_score     16134
home_team_mean_offense_score     15411
home_team_mean_midfield_score    15759
away_team_mean_defense_score     16357
away_team_mean_offense_score     15609
away_team_mean_midfield_score    15942
dtype: int64

## Identificar a partir de quando temos dados de pontuação das seções dos times

In [5]:
nomes_pontuacoes = ["home_team_goalkeeper_score", "away_team_goalkeeper_score", "home_team_mean_defense_score", "home_team_mean_offense_score", "home_team_mean_midfield_score",
"away_team_mean_defense_score", "away_team_mean_offense_score", "away_team_mean_midfield_score"]

columns_has_score = ~data[nomes_pontuacoes].isna()

any_score = columns_has_score.any(axis=1)

In [6]:
elemento_mais_antigo_contem_pontuacao = data[any_score].index.min()
elemento_mais_recente_contem_pontuacao = data[any_score].index.max()

### Partida mais antiga em que foi feita pontuação das seções dos times

In [7]:
data.iloc[[elemento_mais_antigo_contem_pontuacao]]

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
8262,2004-09-03,Maldives,Oman,Africa,Asia,142,50,0,0,1,...,No,Lose,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN


### Partida mais recente em que foi feita pontuação das seções dos times

In [8]:
data.iloc[[elemento_mais_recente_contem_pontuacao]]

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
23920,2022-06-14,Korea Republic,Egypt,Asia,Africa,29,32,1519,1500,4,...,No,Win,75.0,NaN,73.0,80.0,73.8,NaN,79.3,70.8


## Qual time tem os melhores jogadores de defesa, meio-campo e ataque?

In [31]:


colunas_time_casa = ['date', 'home_team', 'home_team_fifa_rank',
       'home_team_total_fifa_points', "home_team_score",
       'home_team_goalkeeper_score', 'home_team_mean_defense_score', 'home_team_mean_offense_score',
       'home_team_mean_midfield_score']

colunas_time_visitante = ['date', 'away_team',  'away_team_fifa_rank', 'away_team_total_fifa_points', 
       'away_team_score', 'away_team_goalkeeper_score',
       'away_team_mean_defense_score', 'away_team_mean_offense_score', 'away_team_mean_midfield_score']

partidas_com_pontuacao = data.iloc[elemento_mais_antigo_contem_pontuacao:elemento_mais_recente_contem_pontuacao]

dados_time_casa = partidas_com_pontuacao[colunas_time_casa]
dados_time_visitante = partidas_com_pontuacao[colunas_time_visitante]

dados_time_casa_renomeado = dados_time_casa.rename(mapper=lambda x: x.replace("home_team_", ""), axis=1)
dados_time_casa_renomeado = dados_time_casa_renomeado.rename(mapper=lambda x: x.replace("home_team", "team"), axis=1)

dados_time_visitante_renomeado = dados_time_visitante.rename(mapper=lambda x: x.replace("away_team_", ""), axis=1)
dados_time_visitante_renomeado = dados_time_visitante_renomeado.rename(mapper=lambda x: x.replace("away_team", "team"), axis=1)


pontuacao_paises = pd.concat([dados_time_casa_renomeado, dados_time_visitante_renomeado], axis=0)

In [33]:
pontuacao_paises["ano"] = pontuacao_paises["date"].dt.year

### Pontuação dos times muda durante o ano?

In [41]:
nomes_pontuacoes = ["mean_defense_score", "mean_offense_score", "mean_midfield_score", "goalkeeper_score"]

var_pontuacao = pontuacao_paises.groupby(["ano", "team"])[nomes_pontuacoes].var()

In [44]:
pontuacoes_variam_no_ano = var_pontuacao.apply(lambda x: x>0)

print(pontuacoes_variam_no_ano.any())

mean_defense_score     True
mean_offense_score     True
mean_midfield_score    True
goalkeeper_score       True
dtype: bool


In [47]:
var_pontuacao[pontuacoes_variam_no_ano.values]

mean_defense_score  mean_offense_score  mean_midfield_score  \
ano  team                                                                    
2005 Albania                  NaN            4.314333             0.058333   
     Albania                  NaN            4.314333             0.058333   
     Algeria               0.0000            0.402381             0.952381   
     Algeria               0.0000            0.402381             0.952381   
     Angola                0.0000            0.818182                  NaN   
...                           ...                 ...                  ...   
2021 Wales                 0.0105            2.362500             0.443625   
     Wales                 0.0105            2.362500             0.443625   
     Wales                 0.0105            2.362500             0.443625   
     Zambia                   NaN            0.504000                  NaN   
     Zimbabwe              0.0000            5.770909             0.000000   

               goalkeeper_score  
ano  team                        
2005 Albania                NaN  
     Albania                NaN  
     Algeria             0.0000  
     Algeria             0.0000  
     Angola              0.0000  
...                         ...  
2021 Wales               0.2625  
     Wales               0.2625  
     Wales               0.2625  
     Zambia                 NaN  
     Zimbabwe               NaN  

[4347 rows x 4 columns]

### Média das pontuações por sessão

In [54]:
media_defesa_pais_ano = pontuacao_paises.groupby(["ano", "team"])["mean_defense_score"].mean()
media_defesa_pais_ano.sort_values(ascending=False)

ano   team      
2004  Italy         91.800000
2005  Italy         91.000000
2004  England       90.500000
2005  England       90.045455
2006  Italy         89.450000
                      ...    
2022  Uzbekistan          NaN
      Vanuatu             NaN
      Vietnam             NaN
      Yemen               NaN
      Zambia              NaN
Name: mean_defense_score, Length: 3550, dtype: float64

In [55]:
media_defesa_pais_ano = pontuacao_paises.groupby(["ano", "team"])["mean_offense_score"].mean()
media_defesa_pais_ano.sort_values(ascending=False)

ano   team             
2004  Netherlands          93.0
      France               93.0
2005  Brazil               92.4
2004  Italy                92.3
      Brazil               92.3
                           ... 
2022  US Virgin Islands     NaN
      Uzbekistan            NaN
      Vanuatu               NaN
      Vietnam               NaN
      Yemen                 NaN
Name: mean_offense_score, Length: 3550, dtype: float64

In [56]:
media_defesa_pais_ano = pontuacao_paises.groupby(["ano", "team"])["mean_midfield_score"].mean()
media_defesa_pais_ano.sort_values(ascending=False)

ano   team    
2004  France      93.200000
      England     91.200000
2005  France      90.909091
2006  Brazil      90.500000
2005  Brazil      90.350000
                    ...    
2022  Vanuatu           NaN
      Vietnam           NaN
      Yemen             NaN
      Zambia            NaN
      Zimbabwe          NaN
Name: mean_midfield_score, Length: 3550, dtype: float64

In [57]:
media_defesa_pais_ano = pontuacao_paises.groupby(["ano", "team"])["goalkeeper_score"].mean()
media_defesa_pais_ano.sort_values(ascending=False)

ano   team      
2004  Italy         97.000
2005  Italy         96.000
      Spain         94.500
2006  Italy         94.375
2004  Spain         94.000
                     ...  
2022  Uzbekistan       NaN
      Vanuatu          NaN
      Yemen            NaN
      Zambia           NaN
      Zimbabwe         NaN
Name: goalkeeper_score, Length: 3550, dtype: float64